# Assignment 3  Dashboard

**Name:** Heriberto Zapata Solano 

**e-mail:** heriberto.zapata7939@alumnos.udg.mx

# MODULES


In [51]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import numpy as np
from scipy.stats import cauchy
from scipy.spatial import distance


- What is "vec2d" used for in this code?

The Vec2d class represents 2D vectors for trajectory calculations, managing x and y coordinates as Brownian motion and random walks. To plot in 3D, create a Vec3d class and update the functions appropriate.

In [52]:
# Clase Vec2d para operaciones vectoriales 2D
class Vec2d:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __add__(self, other):
        return Vec2d(self.x + other.x, self.y + other.y)

    def __mul__(self, scalar):
        return Vec2d(self.x * scalar, self.y * scalar)

# FUNCTIONS



Define functions for simulating Brownian motion, correlated random walks, and Levy flights in 2D. It calculates trajectory lengths, mean squared displacement (MSD), and turning angles.

In [53]:
# Function to generate Brownian Motion
def brownian_motion(steps, speed, start_pos):
    trajectory = [Vec2d(*start_pos)]
    for _ in range(steps):
        angle = np.random.uniform(0, 2 * np.pi)
        step = Vec2d(np.cos(angle), np.sin(angle)) * speed
        trajectory.append(trajectory[-1] + step)
    return trajectory

# Function to generate Correlated Random Walk
def correlated_random_walk(steps, speed, start_pos, cauchy_coeff):
    trajectory = [Vec2d(*start_pos)]
    angle = 0
    for _ in range(steps):
        angle += cauchy.rvs(scale=cauchy_coeff)  # Angle distributed according to Cauchy
        step = Vec2d(np.cos(angle), np.sin(angle)) * speed
        trajectory.append(trajectory[-1] + step)
    return trajectory

# Function to generate Levy Flight
def levy_flight(steps, speed, start_pos, cauchy_coeff, alpha):
    trajectory = [Vec2d(*start_pos)]
    for _ in range(steps):
        direction = np.random.uniform(0, 2 * np.pi)
        step_length = (np.random.pareto(alpha) + 1) * cauchy_coeff
        step = Vec2d(np.cos(direction), np.sin(direction)) * step_length * speed
        trajectory.append(trajectory[-1] + step)
    return trajectory

# Function to calculate the path length
def calculate_path_length(trajectory):
    length = [0]
    for i in range(1, len(trajectory)):
        dist = distance.euclidean((trajectory[i-1].x, trajectory[i-1].y), 
                                  (trajectory[i].x, trajectory[i].y))
        length.append(length[-1] + dist)
    return length

# Function to calculate the Mean Squared Displacement (MSD)
def calculate_msd(trajectory):
    msd = []
    for tau in range(1, len(trajectory)):
        displacements = [(trajectory[i + tau].x - trajectory[i].x)**2 + (trajectory[i + tau].y - trajectory[i].y)**2 
                         for i in range(len(trajectory) - tau)]
        msd.append(np.mean(displacements))
    return msd

# Function to calculate turning angles
def calculate_turning_angles(trajectory):
    angles = []
    for i in range(1, len(trajectory) - 1):
        p1 = np.array([trajectory[i-1].x, trajectory[i-1].y])
        p2 = np.array([trajectory[i].x, trajectory[i].y])
        p3 = np.array([trajectory[i+1].x, trajectory[i+1].y])
        v1 = p2 - p1
        v2 = p3 - p2
        dot_prod = np.dot(v1, v2)
        cross_prod = np.cross(v1, v2)
        angle = np.arctan2(cross_prod, dot_prod)
        angles.append(angle)
    return angles


## how to display the results

Now we use the Dash app that is used for creating interactive, web-based dashboards and data visualizations in Python

What are the advantages:

- Create interactive, easy-to-use data applications or dashboards without learning web development frameworks.


# Display Dash app


This section creates the Dash app, which is used to build the dashboard.

In [54]:
# Create the Dash app
app = dash.Dash(__name__)
app.config.suppress_callback_exceptions = True

In this part defines the visual structure of the dashboard. It uses HTML and Dash components to:

- Create a title (“Assignment 3 HZS”).
- Metrics panel: Includes a drop-down menu to select the type of metric (MSD, Path Length, etc.).
- Parameter control panel: Provides buttons, sliders and input fields to adjust simulation parameters (path type, number of steps, velocity, initial position, etc.).
- Graphics: Includes two graphs to visualize the selected trajectory and metrics.

In [55]:

# Define the layout of the dashboard
app.layout = html.Div([
    html.H1("Assignment 3 HZS", style={'text-align': 'center', 'margin-top': '20px', 'font-size': '32px'}),

    # Metrics panel
    html.Div([
        html.H3("Metrics", style={'text-align': 'center', 'font-size': '20px'}),
        dcc.Dropdown(  # Dropdown to select metric type
            id='metric-type',
            options=[
                {'label': 'Mean Squared Displacement (MSD)', 'value': 'MSD'},
                {'label': 'Path Length', 'value': 'PL'},
                {'label': 'Turning Angle Distribution', 'value': 'TAD'}
            ],
            value='MSD',  # Default value
            style={'margin-bottom': '10px'}
        )
    ], style={
        'position': 'absolute', 'top': '5px', 'left': '1000px', 'width': '300px'  # Position settings
    }),

    # Parameters control panel
    html.Div([
        html.H3("Parameters", style={'text-align': 'center', 'font-size': '20px'}),
        dcc.RadioItems(  # Radio buttons for trajectory type
            id='trajectory-type',
            options=[
                {'label': 'BM', 'value': 'BM'},  # Brownian Motion
                {'label': 'CRW', 'value': 'CRW'},  # Correlated Random Walk
                {'label': 'LF', 'value': 'LF'}  # Lévy Flight
            ],
            value='BM',  # Default value
            labelStyle={'display': 'inline-block', 'margin-right': '10px'}
        ),
        html.Div([
            html.Label("Steps:"),
            dcc.Slider(  # Slider for step selection
                id='steps',
                min=100, max=1000, value=500, step=10,
                tooltip={"placement": "bottom", "always_visible": True},
                marks={100: '100', 1000: '1000'}  # Step marker labels
            )
        ], style={'margin-bottom': '10px'}),
        html.Div([
            html.Label("Speed:"),
            dcc.Input(id='speed', value=1, type='number')  # Input for speed
        ], style={'margin-bottom': '10px'}),
        html.Div([
            html.Label("Initial Position (x, y):"),
            dcc.Input(id='start-x', value=0, type='number', style={'margin-right': '10px'}),  # Initial x position
            dcc.Input(id='start-y', value=0, type='number')  # Initial y position
        ], style={'margin-bottom': '10px'}),
        html.Div([
            html.Label("Cauchy Coefficient:"),
            dcc.Input(id='cauchy-coeff', value=0.7, type='number')  # Input for Cauchy coefficient
        ], style={'margin-bottom': '10px'}),
        html.Div([
            html.Label("Lévy Exponent (α):"),
            dcc.Input(id='levy-exponent', value=2.5, type='number')  # Input for Lévy exponent
        ], style={'margin-bottom': '10px'})
    ], style={
        'position': 'absolute', 'top': '5px', 'left': '5px', 'width': '180px'  # Position settings
    }),

    # Graphs for visualizing data
    html.Div([
        dcc.Graph(id='trajectory-plot', style={'width': '65%', 'height': '500px'}),   # Graph for trajectory
        dcc.Graph(id='metric-plot', style={'width': '100%', 'height': '500px'})  # Graph for metric
    ], style={'margin-left': '200px', 'margin-top': '50px', 'display': 'flex', 'flex-direction': 'row'})
])


Callback to update graphs based on user input: 
- This section contains a callback function that dynamically updates the plots (trajectory and metric) based on user input from the control panel.

Update plots 

- *def update_plots* updates both the trajectory and metric graphs depending on the selected trajectory type, metric type, number of steps, speed, and other parameters.

- Plot Re 

In [56]:
# Callback to update graphs based on user input
@app.callback(
    [Output('metric-plot', 'figure'), Output('trajectory-plot', 'figure')],
    [Input('trajectory-type', 'value'), Input('metric-type', 'value'), Input('steps', 'value'),
     Input('speed', 'value'), Input('start-x', 'value'), Input('start-y', 'value'),
     Input('cauchy-coeff', 'value'), Input('levy-exponent', 'value')]
)
def update_plots(trajectory_type, metric_type, steps, speed, start_x, start_y, cauchy_coeff, levy_exponent):
    # Generate trajectories based on type
    if trajectory_type == 'BM':
        trajectory = brownian_motion(steps, speed, (start_x, start_y))  # Brownian Motion
    elif trajectory_type == 'CRW':
        trajectory = correlated_random_walk(steps, speed, (start_x, start_y), cauchy_coeff)  # Correlated Random Walk
    else:
        trajectory = levy_flight(steps, speed, (start_x, start_y), cauchy_coeff, levy_exponent)  # Lévy Flight

 # Plot trajectory in 3D
    x = [p.x for p in trajectory]
    y = [p.y for p in trajectory]
    z = list(range(len(trajectory)))  # Use step number as z-axis
    trajectory_fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z, mode='lines')])
    trajectory_fig.update_layout(title="3D Trajectory")

    # Plot metric based on selected type
    if metric_type == 'MSD':
        msd = calculate_msd(trajectory)  # Mean Squared Displacement
        metric_fig = go.Figure(data=[go.Scatter(x=list(range(1, len(trajectory))), y=msd, mode='lines')])
        metric_fig.update_layout(title="Mean Squared Displacement (MSD)")
    elif metric_type == 'PL':
        length = calculate_path_length(trajectory)  # Path length
        metric_fig = go.Figure(data=[go.Scatter(x=list(range(len(length))), y=length, mode='lines')])
        metric_fig.update_layout(title="Path Length")
    else:
        angles = calculate_turning_angles(trajectory)  # Turning Angle Distribution
        metric_fig = go.Figure(data=[go.Histogram(x=angles, nbinsx=50)])
        metric_fig.update_layout(title="Turning Angle Distribution")

    return metric_fig, trajectory_fig

 Run app
- The final part of the code runs the app on port 8051 with debugging enabled.

- for a better visualization click on the following link

http://127.0.0.1:8051/ 


In [57]:
# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, port=8051)